In [1]:
import json
from enum import Enum
from collections import namedtuple

In [2]:
class events_list(Enum): 
    Read = 0
    Write = 1
    Acquire = 2
    Release = 3

In [3]:
def _json_object_hook(d): 
    return namedtuple('event_raw', d.keys())(*d.values())

def json2obj(data): 
    return json.loads(data, object_hook=_json_object_hook)

class event_raw:
    def __init__(self):
        self.tid = -1
        self.e_type = 0
        self.var_name = ""
        self.vc = []
        self.vc_r = []
        self.vc_w = []

In [4]:
class critical_section:
    def __init__(self,thread_num):
        self.locks = set()
        self.read_objects = set()
        self.write_objects = set()
        self.child_sections = []
        self.tid = thread_num
        self.event_start = -1
        self.event_end = -1

    def __str__(self):
        ret_str = "Thread : %d \nLocks : %s \nRead Objects : %s \nWrite Objects : %s \nStart : %d \nEnd : %d \nChild Sections :\n" % (self.tid,self.locks,self.read_objects,self.write_objects,self.event_start,self.event_end)
        for child_cs in self.child_sections:
            ret_str= ret_str+str(child_cs)
        return ret_str
    
    def clear_lists(self):
        self.locks.clear()
        self.read_objects.clear()
        self.write_objects.clear()
        self.child_sections.clear()
    
    def create_child_Section(self,evt_obj,thrd_stk,evt_id):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 1:
            if len(temp.child_sections) > 0:
                temp.child_sections[-1].locks.add(evt_obj.var_name)
                temp = temp.child_sections[-1]
            size_stk=size_stk-1
        temp.child_sections.append(critical_section(evt_obj.tid))
        temp.child_sections[-1].locks.add(evt_obj.var_name)
        temp.child_sections[-1].event_start = evt_id
    
    def operate_child(self,evt_obj,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 0:
            temp = temp.child_sections[-1]
            size_stk=size_stk-1
            temp.add_object(evt_obj)
    
    def add_object(self,evt_obj):
        if evt_obj.e_type == events_list.Read.value:
                self.read_objects.add(evt_obj.var_name)
        elif evt_obj.e_type == events_list.Write.value:
                self.write_objects.add(evt_obj.var_name)
    
    def update_event_end(self,end_id,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 0:
            temp = temp.child_sections[-1]
            size_stk=size_stk-1
        temp.event_end = end_id
        
def update_lock_cs_membership(l_map,locks_set,index):
    for x in locks_set:
        if not x in l_map.keys():
            l_map[x] = []    
        l_map[x].append(index)

In [5]:
# # event_log_raw = open("../Figure1.log")
# event_log_raw = open("temp.log")
# # event_log_raw = open("../Test.log")
# event_log = json.loads(event_log_raw.read())

In [6]:
# events = []
# for line in event_log:
#     events.append(json2obj(json.dumps(line)))

In [7]:
def gen_event_cs(filename):
    event_log_raw = open(filename)
    event_log = json.loads(event_log_raw.read())

    events = []
    for line in event_log:
        events.append(json2obj(json.dumps(line)))

    thread_cs = []
    critical_section_list = []
    thread_lock_stack = []
    lock_map = {}
    ct = 0
    for x in events:
        if(len(thread_cs)<x.tid+1):
            while(len(thread_cs)!=x.tid+1):
                thread_cs.append(None)
                thread_lock_stack.append([])
        if thread_cs[x.tid]==None: # Start of new crititcal section
            thread_cs[x.tid] = critical_section(x.tid)
            thread_cs[x.tid].clear_lists()
            thread_cs[x.tid].event_start = ct
            if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
    #             print("Here")
                thread_cs[x.tid].add_object(x)#.objects.add(x.var_name)
                thread_cs[x.tid].event_end = ct
                critical_section_list.append(thread_cs[x.tid])
                thread_cs[x.tid] = None
            elif x.e_type == events_list.Acquire.value:
                thread_cs[x.tid].locks.add(x.var_name)
                thread_lock_stack[x.tid].append(x.var_name)
        else:
            if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
                if len(thread_lock_stack[x.tid]) > 1:
                    thread_cs[x.tid].operate_child(x,thread_lock_stack[x.tid])
                thread_cs[x.tid].add_object(x)#.objects.add(x.var_name)
            elif x.e_type == events_list.Acquire.value:
                if len(thread_lock_stack[x.tid]) == 0:
                    thread_cs[x.tid].event_end = ct-1
                    critical_section_list.append(thread_cs[x.tid])
                    thread_cs[x.tid] = None
                    thread_cs[x.tid] = critical_section(x.tid)
                    thread_cs[x.tid].clear_lists()
                    thread_cs[x.tid].event_start = ct
                    thread_lock_stack[x.tid].append(x.var_name)
                elif thread_lock_stack[x.tid][-1] != x.var_name:
                    thread_lock_stack[x.tid].append(x.var_name)
                    thread_cs[x.tid].create_child_Section(x,thread_lock_stack[x.tid],ct)
                thread_cs[x.tid].locks.add(x.var_name)
            elif x.e_type == events_list.Release.value:
                thread_cs[x.tid].update_event_end(ct,thread_lock_stack[x.tid])
                if len(thread_lock_stack[x.tid]) == 1:
    #                 print(thread_cs[x.tid-1])
                    critical_section_list.append(thread_cs[x.tid])
                    update_lock_cs_membership(lock_map,thread_cs[x.tid].locks,len(critical_section_list)-1)
                    thread_cs[x.tid] = None
                # Note: Optimistic assumption, top of the lock is same
                if thread_lock_stack[x.tid][-1] == x.var_name:
                    thread_lock_stack[x.tid].pop()
#         print(thread_cs[x.tid])
        ct+=1
    return events,critical_section_list,len(thread_cs),lock_map

In [8]:
e,cs,l,lock_mem = gen_event_cs("temp.log")

In [9]:
class critical_section2:
    def __init__(self,thread_num):
        self.locks = set()
        self.read_objects = set()
        self.write_objects = set()
        self.child_sections = set()
        self.tid = thread_num
        self.event_idx = set()

    def __str__(self):
        ret_str = "Thread : %d \nLocks : %s \nRead Objects : %s \nWrite Objects : %s \nEvents : %s \nChild Sections : %s\n" % (self.tid,self.locks,self.read_objects,self.write_objects,self.event_idx,self.child_sections)
        return ret_str
    
    def clear_lists(self):
        self.locks.clear()
        self.read_objects.clear()
        self.write_objects.clear()
        self.child_sections.clear()
    
    def merge(self,child):
        self.locks = self.locks.union(child.locks)
        self.read_objects = self.read_objects.union(child.read_objects)
        self.write_objects = self.write_objects.union(child.write_objects)
        self.event_idx = self.event_idx.union(child.event_idx)
        self.child_sections = self.child_sections.union(child.child_sections)
        
    def add_object(self,evt_obj):
        if evt_obj.e_type == events_list.Read.value:
                self.read_objects.add(evt_obj.var_name)
        elif evt_obj.e_type == events_list.Write.value:
                self.write_objects.add(evt_obj.var_name)

In [10]:
def gen_event_cs2(filename):
    event_log_raw = open(filename)
    event_log = json.loads(event_log_raw.read())

    events = []
    for line in event_log:
        events.append(json2obj(json.dumps(line)))

    thread_cs = []
    critical_section_list = []
    thread_cs_stack = []
    lock_map = {}
    ct = 0
    for x in events:
        if(len(thread_cs)<x.tid+1):
            while(len(thread_cs)!=x.tid+1):
                thread_cs.append(None)
                thread_cs_stack.append([])
        if thread_cs[x.tid]==None: # Start of new crititcal section
    #         print("Starting new critial section : ")
            thread_cs[x.tid] = critical_section2(x.tid)
            thread_cs[x.tid].clear_lists()
            thread_cs[x.tid].event_idx.add(ct)
            if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
    #             print("Here")
                thread_cs[x.tid].add_object(x)
                critical_section_list.append(thread_cs[x.tid])
                thread_cs[x.tid] = None
            elif x.e_type == events_list.Acquire.value:
                thread_cs[x.tid].locks.add(x.var_name)
                critical_section_list.append(thread_cs[x.tid])
                thread_cs_stack[x.tid].append(len(critical_section_list)-1)
                update_lock_cs_membership(lock_map,thread_cs[x.tid].locks,len(critical_section_list)-1)
        else:
            if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
                thread_cs[x.tid].add_object(x)
                thread_cs[x.tid].event_idx.add(ct)
            elif x.e_type == events_list.Acquire.value:
                critical_section_list[thread_cs_stack[x.tid][-1]] = thread_cs[x.tid]
                thread_cs[x.tid] = critical_section2(x.tid)
                thread_cs[x.tid].locks.add(x.var_name)
                thread_cs[x.tid].event_idx.add(ct)
                critical_section_list.append(thread_cs[x.tid])
                thread_cs_stack[x.tid].append(len(critical_section_list)-1)
                update_lock_cs_membership(lock_map,thread_cs[x.tid].locks,len(critical_section_list)-1)
            elif x.e_type == events_list.Release.value:
                thread_cs[x.tid].event_idx.add(ct)
                cs_idx = thread_cs_stack[x.tid].pop()
                critical_section_list[cs_idx] = thread_cs[x.tid]
                if len(thread_cs_stack[x.tid]) > 0:
                    thread_cs[x.tid] = critical_section_list[thread_cs_stack[x.tid][-1]]
                    thread_cs[x.tid].merge(critical_section_list[cs_idx])
                    thread_cs[x.tid].child_sections.add(cs_idx)
                else:
                    thread_cs[x.tid] = None
#         print(thread_cs[x.tid])
        ct+=1
    return events,critical_section_list,len(thread_cs),lock_map

In [11]:
e,csl,l,m = gen_event_cs2("temp.log")

In [12]:
for x in csl:
    print(x)

Thread : 1 
Locks : set() 
Read Objects : {'phi_x'} 
Write Objects : set() 
Events : {0} 
Child Sections : set()

Thread : 1 
Locks : set() 
Read Objects : {'phi_y'} 
Write Objects : set() 
Events : {1} 
Child Sections : set()

Thread : 1 
Locks : {'m', 'n', 'l'} 
Read Objects : {'xl', 'x', 'a'} 
Write Objects : {'yl', 'b', 'y'} 
Events : {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17} 
Child Sections : {3, 4, 5}

Thread : 1 
Locks : {'n', 'l'} 
Read Objects : {'xl', 'x'} 
Write Objects : {'yl', 'y'} 
Events : {5, 6, 7, 8, 9, 10, 11, 12} 
Child Sections : {4}

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {8, 9, 10, 11} 
Child Sections : set()

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {16, 13, 14, 15} 
Child Sections : set()

